In [1]:
%matplotlib inline

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import scipy.interpolate as interp
from astropy import units as u
from astropy import constants as const
from tqdm import tqdm

Run IMF sampling for masses that are relevant for the isochrones used for fitting with many mnay stars.


In [2]:
path_plot = '/users/caganze/research/stellarstreams/figures/isochrones/'
path_data = '/users/caganze/research/stellarstreams/data/isochrones/'

In [ ]:
dist_mod_Pal5 = 16.8553 #Pal 5
#dist_mod_And = 16.8553 + 7.6 #Andromeda 
dist_mod_And = 24.47 #from Ibata+2014

wfirst_mini = np.loadtxt(path_data+'/WFIRST_PARSEC_isochrones_grid.txt', skiprows = 12, usecols = [2])
wfirst_mags = np.loadtxt(path_data+'/WFIRST_PARSEC_isochrones_grid.txt', skiprows = 12, usecols = [10,11,12,13,14,15])
isos = np.loadtxt(path_data+'/WFIRST_PARSEC_isochrones_grid.txt', skiprows = 12, usecols = [0,1])
all_iso = []
start_feh = [0,0,0,0]
start_age = np.repeat(0,16)
j_feh = 1
j_age = 1
for i in range(len(isos[:,0])-1):
    if (isos[i+1,0] != isos[i,0]):
        start_feh[j_feh] = i+1
        j_feh = j_feh + 1
    if (isos[i+1,1] != isos[i,1]):
        start_age[j_age] = i+1
        j_age = j_age + 1

for i in range(len(start_age)):
#for i in range(1):
    start_isochrone = start_age[i]
    if i == len(start_age)-1:
        end_isochrone = len(wfirst_mini)-1
    else:
        end_isochrone = start_age[i+1]-1
    inter_Rmags = interp.interp1d(wfirst_mini[start_isochrone:end_isochrone], 
                                  wfirst_mags[start_isochrone:end_isochrone,0])
    inter_Zmags = interp.interp1d(wfirst_mini[start_isochrone:end_isochrone], 
                                  wfirst_mags[start_isochrone:end_isochrone,1])
    inter_Ymags = interp.interp1d(wfirst_mini[start_isochrone:end_isochrone], 
                                  wfirst_mags[start_isochrone:end_isochrone,2])
    inter_Jmags = interp.interp1d(wfirst_mini[start_isochrone:end_isochrone], 
                                  wfirst_mags[start_isochrone:end_isochrone,3])
    inter_Hmags = interp.interp1d(wfirst_mini[start_isochrone:end_isochrone], 
                                  wfirst_mags[start_isochrone:end_isochrone,4])
    inter_Fmags = interp.interp1d(wfirst_mini[start_isochrone:end_isochrone], 
                                  wfirst_mags[start_isochrone:end_isochrone,5])

    #sample from IMF sub-region renormalize to the right edges.
    class imf_sub(st.rv_continuous):
        def _pdf(self,x):
            return 1.0/(2*np.sqrt(np.max(wfirst_mini[start_isochrone:end_isochrone]))-2*np.sqrt(np.min(wfirst_mini[start_isochrone:end_isochrone])))*x**(-0.5)  # Normalized over its range, in this case [0.01,120]


    my_imf_low = imf_sub(a=np.min(wfirst_mini[start_isochrone:end_isochrone]), 
                         b=np.max(wfirst_mini[start_isochrone:end_isochrone]), name='sub')
    
    num = 1000000
    sample_lowmasses = my_imf_low.rvs(size=num)
    sample_Rmags = inter_Rmags(sample_lowmasses)
    sample_Zmags = inter_Zmags(sample_lowmasses)
    sample_Ymags = inter_Ymags(sample_lowmasses)
    sample_Jmags = inter_Jmags(sample_lowmasses)
    sample_Hmags = inter_Hmags(sample_lowmasses)
    sample_Fmags = inter_Fmags(sample_lowmasses)
    lumfunc_R, R_edges = np.histogram(sample_Rmags, range = [np.min(sample_Rmags), np.max(sample_Rmags)], bins = 100)
    lumfunc_Z, Z_edges = np.histogram(sample_Zmags, range = [np.min(sample_Zmags), np.max(sample_Zmags)], bins = 100)
    lumfunc_Y, Y_edges = np.histogram(sample_Ymags, range = [np.min(sample_Ymags), np.max(sample_Ymags)], bins = 100)
    lumfunc_J, J_edges = np.histogram(sample_Jmags, range = [np.min(sample_Jmags), np.max(sample_Jmags)], bins = 100)
    lumfunc_H, H_edges = np.histogram(sample_Hmags, range = [np.min(sample_Hmags), np.max(sample_Hmags)], bins = 100)
    lumfunc_F, F_edges = np.histogram(sample_Fmags, range = [np.min(sample_Fmags), np.max(sample_Fmags)], bins = 100)
    lum_R = R_edges[:-1]+0.5*(R_edges[1:]-R_edges[:-1])
    lum_Z = Z_edges[:-1]+0.5*(Z_edges[1:]-Z_edges[:-1])
    lum_Y = Y_edges[:-1]+0.5*(Y_edges[1:]-Y_edges[:-1])
    lum_J = J_edges[:-1]+0.5*(J_edges[1:]-J_edges[:-1])
    lum_H = H_edges[:-1]+0.5*(H_edges[1:]-H_edges[:-1])
    lum_F = F_edges[:-1]+0.5*(F_edges[1:]-F_edges[:-1])
    np.savetxt('WFIRST_PARSEC_lumfunc_isochrone%d_allbands.txt' %i, 
               np.concatenate((np.reshape(lum_R, (len(lum_R),1)), np.reshape(lumfunc_R, (len(lum_R),1)), 
                               np.reshape(lum_Z, (len(lum_Z),1)), np.reshape(lumfunc_Z, (len(lum_Z),1)),
                               np.reshape(lum_Y, (len(lum_Y),1)), np.reshape(lumfunc_Y, (len(lum_Y),1)),
                               np.reshape(lum_J, (len(lum_J),1)), np.reshape(lumfunc_J, (len(lum_J),1)),
                               np.reshape(lum_H, (len(lum_H),1)), np.reshape(lumfunc_H, (len(lum_H),1)),
                               np.reshape(lum_F, (len(lum_F),1)), np.reshape(lumfunc_F, (len(lum_F),1))
                              ), axis = 1), 
               header = 'lumR, num, lumZ, num, lumY, num, lumJ, num, lumH, num, lumF, num; total number of stars in each luminosity function = %d' %num)
    print ("done", i, "of", len(start_age))
#fig, ax = plt.subplots(1,1, figsize=[6,4])
#ax.set_ylabel('number')
#ax.set_xlabel('R')
#ax.set_title('luminosity function from IMF sampling sample')
#h, be, p = ax.hist(sample_Rmags, bins = 100, range = [np.min(sample_Rmags), np.max(sample_Rmags)])  

done 0 of 16
